In [ ]:
# latest update
# June 27, 2017
# Changes done by: Jimmy Vivas

# This code was developed by Jimmy Vivas for Aspen Technology
# For Shell case, they created one file per month and a folder per tag

# The code will take all files within sub folders but it wont process files located in the root folder
# The colde will take the file name as a tagname and look for that tagname in a tag dictionary file
# Then it will add the tagname and tag description to the CSV file for each file wihtin sub folders
# The code will merge all individual csv files (monthly files) to be one single file

In [1]:
from os import listdir
from os import walk
import pandas as pd
import math
import numpy as np
#disabling chained mode to be able to update datetime with fixed data
pd.options.mode.chained_assignment = None  # default='warn'

In [10]:
# mac Path
#CSVPath1 =  '/Users/jvivas/Documents/XMT Baytwon Sensor Data' \
#            '/Sensor Data/Testing'

CSVPath1 = 'C:/Users/vivasj/Documents/Aspen/ALCOA/data/WAC Minute Data 2/to be processed'
CSVMetaDataFile = 'C:/Users/vivasj/Documents/Aspen/ALCOA/data/WAC Minute Data 2/MetaData/Data Dictionary.csv'
CSVPathProcessed = 'C:/Users/vivasj/Documents/Aspen/ALCOA/data/WAC Minute Data 2/Processed'

# Windows Path
# CSVPath1 = 'C:/Users/jvivas/Dropbox/Mtell Customer Projects/XOM BayTown RHC MEA Tower Foaming/Sensor Data/' \
#           'To be Processed'

# CSVPath1 = 'C:/Mtell/Projects/XOM Baytown POC/Sensor Data/ToBeFormatted'

In [3]:
def GetCSV(CSVPath):
    CSVFileList = []
    NanPHDTagList = []
    CSVFileListAll = listdir(CSVPath)

    m = len(CSVFileListAll)

    for i in range(m):
        fileNameStr = CSVFileListAll[i]
        fileStr = fileNameStr[:-4]
        fileExt = fileNameStr[-3:]
        if fileExt == "csv":
            CSVFileList.append(fileNameStr)

    n = len(CSVFileList)
    return(n, CSVFileList)

In [4]:
def GetExcels(CSVPath):
    CSVFileList = []
    NanPHDTagList = []
    CSVFileListAll = listdir(CSVPath)

    m = len(CSVFileListAll)

    for i in range(m):
        fileNameStr = CSVFileListAll[i]
        fileStr = fileNameStr[:-5]
        fileExt = fileNameStr[-4:]
        if fileExt == "xlsx":
            CSVFileList.append(fileNameStr)

    n = len(CSVFileList)
    return(n, CSVFileList)

In [5]:
def LoadCSV(FileAndPath, file):
    print ('Loading ' + file)
    df2 = pd.read_csv(FileAndPath+'/'+file, low_memory=False, header=None)
    df2.columns = ['DATETIME','VALUE']
    return(df2)

In [6]:
def GetTagName(CSVFileName):
    
    TagName = CSVFileName[:CSVFileName.find('-')]
    return(TagName)

In [7]:
def LoadExcel(FileAndPath, file):
    
    print('Loading Excel File ' + file)
    xls = pd.ExcelFile(FileAndPath + '/' + file, low_memory=False)
    print('Loading multiple sheets')
    df1_2014 = pd.read_excel(xls, '2014',names=['DATETIME','VALUE'],header=None)
    df1_2015 = pd.read_excel(xls, '2015',names=['DATETIME','VALUE'],header=None)
    df1_2016 = pd.read_excel(xls, '2016',names=['DATETIME','VALUE'],header=None)
    df1_2017 = pd.read_excel(xls, '2017',names=['DATETIME','VALUE'],header=None)

    frames = [df1_2014, df1_2015, df1_2016]

    result = pd.concat(frames)
    print('Done importing Excel file')
    return(result)

# This is the Main Code to execute

In [11]:
print ('Loading Tagname Metadata')
TagMetaDataDF = pd.read_csv(CSVMetaDataFile, low_memory=False)

CSVFinalPath = CSVPath1
n, CSVFileList = GetCSV(CSVFinalPath)

i = 0
for i in range(n):
    CSVFile = CSVFileList[i]
    # CSVFileWithPath = CSVPath1 + "/" + CSVFileList[i]

    df2 = LoadCSV(CSVFinalPath, CSVFile)

    print ('Getting TagName')
    TagName = GetTagName(CSVFile)
    
    print('Getting TagDescription')
    TagDescription = TagMetaDataDF.loc[TagMetaDataDF['TagName'] == TagName, 'TagDescription'].values[0]
    
    print ('Adding Tag name and description to CSV file')

    df2['TagName'] = TagName
    df2['TagDescription'] = TagDescription
    
    for index, row in df2[np.column_stack([df2['DATETIME'].str.contains(r"PM|AM", na=False)])].iterrows():
        BadDateStr = df2[np.column_stack([df2['DATETIME'].str.contains(r"PM|AM", na=False)])]['DATETIME'][index]
        IndexFirstSlash = BadDateStr.find("/")
        IndexSecondSlash = BadDateStr.find("/",IndexFirstSlash+1)
        # print(IndexFirstSlash)
        # print(IndexSecondSlash)
        # print(BadDateStr[:IndexFirstSlash])
        # print(BadDateStr[IndexFirstSlash+1:IndexSecondSlash])
        Len = len(BadDateStr)
        # print(len(BadDateStr))
        # print(BadDateStr[IndexSecondSlash:Len])
        # print(BadDateStr[IndexFirstSlash+1:IndexSecondSlash]+'/'+BadDateStr[:IndexFirstSlash]+'/'+BadDateStr[IndexSecondSlash+1:Len])
        df2['DATETIME'][index] = BadDateStr[IndexFirstSlash+1:IndexSecondSlash]+'/'+BadDateStr[:IndexFirstSlash]+'/'+BadDateStr[IndexSecondSlash+1:Len]
    
    df2.to_csv(CSVPathProcessed + '/' + str(CSVFile.replace('.csv', '')) + '_Formatted.csv', index=False)
    print ('Tagname: ' + TagName + ' Processed')
print ('ALL DONE')

Loading Tagname Metadata
Loading CLP.RCFTWC22.PV-2012.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.RCFTWC22.PV Processed
Loading CLP.RCFTWC22.PV-2013.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.RCFTWC22.PV Processed
Loading CLP.RCFTWC22.PV-2014.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.RCFTWC22.PV Processed
Loading CLP.RCFTWC22.PV-2015.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.RCFTWC22.PV Processed
Loading CLP.RCFTWC22.PV-2016.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.RCFTWC22.PV Processed
Loading CLP.RCFTWC22.PV-2017.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.RCFTWC22.PV Processed
Loading CLP.RCWUFC22.PV-2012.csv
Getting TagName
Getting TagDescripti

Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.TI35J12.PV Processed
Loading CLP.TI35J12.PV-2013.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.TI35J12.PV Processed
Loading CLP.TI35J12.PV-2014.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.TI35J12.PV Processed
Loading CLP.TI35J12.PV-2015.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.TI35J12.PV Processed
Loading CLP.TI35J12.PV-2016.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.TI35J12.PV Processed
Loading CLP.TI35J12.PV-2017.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.TI35J12.PV Processed
Loading CLP.TISLTF11.PV-2012.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.TISLTF11.PV Processed
Loa

Tagname: CLP.WIMTWC24.PV Processed
Loading CLP.WIMTWC24.PV-2014.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.WIMTWC24.PV Processed
Loading CLP.WIMTWC24.PV-2015.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.WIMTWC24.PV Processed
Loading CLP.WIMTWC24.PV-2016.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.WIMTWC24.PV Processed
Loading CLP.WIMTWC24.PV-2017.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.WIMTWC24.PV Processed
Loading CLP.YSB35C21.PVN.1DYAVG-2012.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.YSB35C21.PVN.1DYAVG Processed
Loading CLP.YSB35C21.PVN.1DYAVG-2013.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: CLP.YSB35C21.PVN.1DYAVG Processed
Loading CLP.YSB35C21.PVN.1D

Tagname: DGP.TIH21.PV Processed
Loading DGP.TIH21A.PV-2012.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: DGP.TIH21A.PV Processed
Loading DGP.TIH21A.PV-2013.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: DGP.TIH21A.PV Processed
Loading DGP.TIH21A.PV-2014.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: DGP.TIH21A.PV Processed
Loading DGP.TIH21A.PV-2015.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: DGP.TIH21A.PV Processed
Loading DGP.TIH21A.PV-2016.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: DGP.TIH21A.PV Processed
Loading DGP.TIH21A.PV-2017.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: DGP.TIH21A.PV Processed
Loading WAC2_CTL._CV36.READVALUE-2014.csv
Getting TagName
Getting TagDescription
Addin

Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._CV46.READVALUE Processed
Loading WAC2_CTL._CV47.READVALUE-2014.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._CV47.READVALUE Processed
Loading WAC2_CTL._CV47.READVALUE-2015.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._CV47.READVALUE Processed
Loading WAC2_CTL._CV47.READVALUE-2016.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._CV47.READVALUE Processed
Loading WAC2_CTL._CV47.READVALUE-2017.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._CV47.READVALUE Processed
Loading WAC2_CTL._CV48.READVALUE-2014.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._CV48.READVALUE Processed
Loading WAC2_CTL._CV48.READV

Tagname: WAC2_CTL._CV58.READVALUE Processed
Loading WAC2_CTL._CV59.READVALUE-2014.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._CV59.READVALUE Processed
Loading WAC2_CTL._CV59.READVALUE-2015.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._CV59.READVALUE Processed
Loading WAC2_CTL._CV59.READVALUE-2016.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._CV59.READVALUE Processed
Loading WAC2_CTL._CV59.READVALUE-2017.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._CV59.READVALUE Processed
Loading WAC2_CTL._CV60.READVALUE-2014.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._CV60.READVALUE Processed
Loading WAC2_CTL._CV60.READVALUE-2015.csv
Getting TagName
Getting TagDescription
Adding Tag name and descriptio

Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._MV1.READVALUE Processed
Loading WAC2_CTL._MV1.READVALUE-2017.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._MV1.READVALUE Processed
Loading WAC2_CTL._MV10.READVALUE-2014.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._MV10.READVALUE Processed
Loading WAC2_CTL._MV10.READVALUE-2015.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._MV10.READVALUE Processed
Loading WAC2_CTL._MV10.READVALUE-2016.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._MV10.READVALUE Processed
Loading WAC2_CTL._MV10.READVALUE-2017.csv
Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._MV10.READVALUE Processed
Loading WAC2_CTL._MV11.READVALU

Getting TagName
Getting TagDescription
Adding Tag name and description to CSV file
Tagname: WAC2_CTL._MV9.READVALUE Processed
